# Deskripsi dataset:
Saat ini, kami memiliki 200 entri data demografi pelanggan supermarket. Berikut adalah gambaran singkat tentang kolom dataset:

1. CustomerID: ID pelanggan yang diindeks oleh sistem keanggotaan
2. Gender: jenis kelamin pelanggan supermarket
3. Age: usia anggota supermarket
4. Annual Income (k$): pendapatan tahunan pelanggan dengan satuan ribu dolar
5. Spending Score (1-100): skor pengeluaran dari 1-100. 100 berarti pelanggan adalah pembeli yang aktif.

In [2]:
#Import Modul
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go

### 1. Baca file CSV dengan Pandas.

In [3]:
customer_df = pd.read_csv("Mall_Customers.csv")
customer_df.head()


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [4]:
# shape of the dataset
print(customer_df.shape)

(200, 5)


Apakah ada data yang hilang?

In [5]:
# Fungsi reusable pribadi saya untuk mendeteksi data yang hilang
def missing_value_describe(data):
# Periksa nilai yang hilang dalam data
    missing_value_stats = (data.isnull().sum() / len(data)*100)
    missing_value_col_count = sum(missing_value_stats > 0)
    missing_value_stats = missing_value_stats.sort_values(ascending=False)[:missing_value_col_count]
    print("Jumlah kolom dengan nilai yang hilang:", missing_value_col_count)
    if missing_value_col_count != 0:
        # Mencetak nama kolom dengan persentase nilai yang hilang
        print("\nPersentase hilang (menurun):")
        print(missing_value_stats)
    else:
        print("Tidak ada data yang hilang!!!")
missing_value_describe(customer_df)

Jumlah kolom dengan nilai yang hilang: 0
Tidak ada data yang hilang!!!


Okey, Bagus! 

jadi tidak memiliki data yang hilang.

### 2. Analisis data eksploratori dari data anggota

#### 2.0 Bagaimana distribusi skor pengeluaran?

In [6]:
px.histogram(customer_df, y="Spending Score (1-100)", 
             marginal="rug", title="Spending score pengeluaran")

Observasi:
Dari histogram distribusi skor pengeluaran yang dikemas dengan penambahan 10, kita dapat melihat ada 3 puncak di sekitar skor pengeluaran 10-19, 40-49, dan 70-79 dengan jumlah pelanggan maksimal sebesar 39 jatuh ke dalam rentang skor pengeluaran 40-49. Data ini berpusat di sekitar rentang skor pengeluaran 40-49.

#### 2.1 Bagaimana distribusi skor pengeluaran dari pelanggan pria dan wanita?

In [7]:
customer_df.Gender.value_counts()

Female    112
Male       88
Name: Gender, dtype: int64

In [8]:
px.box(customer_df, x="Spending Score (1-100)", y="Gender",
          color="Gender", points='all',
          title="Distribusi skor pengeluaran berdasarkan gender")

In [9]:
# Ringkasan statistik skor pengeluaran pelanggan pria
customer_df.query("Gender == 'Male'")["Spending Score (1-100)"].describe()

count    88.000000
mean     48.511364
std      27.896770
min       1.000000
25%      24.500000
50%      50.000000
75%      70.000000
max      97.000000
Name: Spending Score (1-100), dtype: float64

In [10]:
# Ringkasan statistik skor pengeluaran pelanggan wanita

customer_df.query("Gender == 'Female'")["Spending Score (1-100)"].describe()

count    112.000000
mean      51.526786
std       24.114950
min        5.000000
25%       35.000000
50%       50.000000
75%       73.000000
max       99.000000
Name: Spending Score (1-100), dtype: float64

<b>Observasi:</b>
 
Dari visualisasi di atas, kita dapat mengamati bahwa tidak ada perbedaan yang jelas antara median dan kuartil atas dari skor pengeluaran pelanggan pria dan wanita. Kuartil pertama dari skor pengeluaran pria lebih rendah sebesar 10,5 unit dibandingkan dengan kuartil pertama dari skor pengeluaran wanita. Skor pengeluaran maksimum (99) untuk wanita lebih tinggi dari skor pengeluaran maksimum (97) untuk pria. Skor pengeluaran minimum (5) untuk wanita lebih tinggi dari skor pengeluaran minimum (1) untuk pria.


#### 2.2 Bagaimana distribusi skor pengeluaran dari kelompok usia yang berbeda?

In [11]:
px.histogram(customer_df, x="Age", title="Distribusi usia pelanggan", nbins=10)


<b>Observasi:</b>

Distribusi data usia pelanggan relatif normal dengan berpusat pada rentang usia 30-39 dengan total jumlah 61 pelanggan.


In [12]:
customer_df.Age.describe()

count    200.000000
mean      38.850000
std       13.969007
min       18.000000
25%       28.750000
50%       36.000000
75%       49.000000
max       70.000000
Name: Age, dtype: float64

In [13]:
# Buat kolom baru kelompok usia dengan penambahan 10 unit
def bin_age(age):
    if age < 20: return "10-19"
    elif age < 30: return "20-29"
    elif age < 40: return "30-39"
    elif age < 50: return "40-49"
    elif age < 60: return "50-49"
    elif age < 70: return "60-69"
    else: return "70-79"

customer_df["age_group"] = customer_df.Age.apply(bin_age)


In [14]:
customer_df["age_group"].unique()

array(['10-19', '20-29', '30-39', '60-69', '50-49', '40-49', '70-79'],
      dtype=object)

In [20]:
age_group_order = ['10-19', '20-29', '30-39', '40-49', '50-49', '60-69', '70-79']
px.box(customer_df, y="Spending Score (1-100)", x="age_group",
       title="Distribusi skor pengeluaran berdasarkan kelompok umur dan kelompok gender",
       color="Gender",
       category_orders={"age_group": age_group_order},
       facet_row="Gender")

<b>Observation:</b>

Dari plot kotak di atas, kita dapat mengamati bahwa distribusi kedua kelompok gender memiliki pola yang sama untuk dua kelompok usia 20-29 dan 30-39. Skor pengeluaran rata-rata 30-39 adalah yang tertinggi dibandingkan dengan kelompok usia lainnya.

#### 2.3 Apa distribusi pendapatan tahunan dari berbagai kelompok umur?

In [21]:
age_group_order = ['10-19', '20-29', '30-39', '40-49', '50-49', '60-69', '70-79']
px.box(customer_df, y="Annual Income (k$)", x="age_group",
       title="Distribusi pendapatan tahunan berdasarkan kelompok umur dan kelompok gender",
       color="Gender",
       category_orders={"age_group": age_group_order},
       facet_row="Gender")

<b>Observation:</b>

Dari plot kotak di atas, kita dapat mengamati bahwa pendapatan tahunan rata-rata tertinggi untuk pelanggan pria adalah 78K untuk kelompok usia 30-39 dan pendapatan tahunan medina tertinggi untuk pelanggan wanita adalah 74K untuk kelompok usia 30-39. Kelompok usia 30-39 juga merupakan kelompok usia dengan skor pengeluaran tertinggi dari plot kotak di bagian sebelumnya.